In [1]:
%pip install qiskit==1.2.4
%pip install qiskit-aer==0.15.1
%pip install pylatexenc==2.10


[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
#2833792m

In [3]:
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister, transpile
from qiskit.converters import circuit_to_gate
from qiskit.visualization import array_to_latex
from qiskit.quantum_info import Operator
from qiskit.quantum_info import Statevector
from qiskit import transpile
from qiskit.providers.basic_provider import BasicSimulator
from qiskit.visualization import plot_histogram
from qiskit.circuit import ControlledGate
import math
import numpy as np

# The aim of the assignment is to simulate the Ekert91 key distribution protocol.

# This notebook is for a simulation of the protocol without an attacker.


N = 100
rounds = (9 * N) // 2


root2 = math.sqrt(2)
denom1 = math.sqrt(4 + 2 * root2)
denom2 = math.sqrt(4 - 2 * root2)

#A2 = B1 = W = 1/sqrt(2)(X+Z)
#B3 = V = 1/sqrt(2)(X-Z)
W_transform_matrix = [[-1 / denom1, (1 + root2) / denom1],
                      [1 / denom2, (root2 - 1) / denom2]]
W_transform = Operator(W_transform_matrix)

V_transform_matrix = [[1 / denom1, (1 + root2) / denom1],
                      [-1 / denom2, (root2 - 1) / denom2]]
V_transform = Operator(V_transform_matrix)

def random_basis():
    qc = QuantumCircuit(1, 1)
    qc.initialize([1/np.sqrt(3), np.sqrt(2)/np.sqrt(3)], 0)
    qc.measure(0, 0)

    backend = BasicSimulator()
    compiled = transpile(qc, backend)
    job = backend.run(compiled, shots=1, memory=True)
    val = job.result().get_memory()[0] #0 or 1

    if val == "0":
        return "X" #1/3
    else:
        return np.random.choice(["W", "Z"],p=[0.5, 0.5])  #2/3

def entangledPair():
    qc = QuantumCircuit(2,2)
    qc.h(0) #superposition 1/sqrt(2) * (|00⟩ + |10⟩)
    qc.cx(0,1) # 1/sqrt(2) * (|00⟩ + |11⟩)
    qc.x(1)  #1/sqrt(2) * (|01⟩ + |10⟩)
    qc.z(0)  #1/sqrt(2) * (|01⟩ - |10⟩)
    return qc


def measuring(qc, qubit, bit, basis):
    #A1 = X, A2 = W, A3 = Z
    #B1 = W, B2 = Z, B3 = V in coursework notes
    if basis == "X":
        qc.h(qubit)
    elif basis == "W":
        qc.unitary(W_transform_matrix, [qubit])
    elif basis == "V":
        qc.unitary(V_transform_matrix, [qubit])
    qc.measure(qubit, bit)


def average(c,n):
    backend = BasicSimulator()
    compiled = transpile(c, backend)
    job_sim = backend.run(compiled, shots=n)
    result_sim = job_sim.result()
    counts = result_sim.get_counts(compiled)
    #print(counts)
    count00 = counts.get("00",0)
    count01 = counts.get("01",0)
    count10 = counts.get("10",0)
    count11 = counts.get("11",0)
    # The return value includes the conversion from measurement results 0,1 to +1,-1
    # Each 00 means a value of  1 (+1 * +1)
    # Each 01 means a value of -1 (+1 * -1)
    # Each 10 means a value of -1 (-1 * +1)
    # Each 11 means a value of  1 (+1 * +1)
    return (count00 - count01 - count10 + count11) / n

def bell_inequality(rounds):
    results = []
    basis_pairs = [("X", "W"), ("X", "V"), ("Z", "W"), ("Z", "V")]

    for alice_basis, bob_basis in basis_pairs:
        qc = entangledPair()
        measuring(qc, 0, 0, alice_basis)
        measuring(qc, 1, 1, bob_basis)
        results.append(average(qc, rounds))  

    S = abs(results[0] - results[1] + results[2] + results[3])
    print(f"S = {S:.5f}")
    return S



def key(rounds):
    key_alice = []
    key_bob = []
    shared_basis = [("W", "W"), ("Z", "Z")] 

    for _ in range(rounds):
        alice_basis = random_basis()
        bob_basis = random_basis()

        qc = entangledPair()
        measuring(qc, 0, 0, alice_basis)
        measuring(qc, 1, 1, bob_basis)

        backend = BasicSimulator()
        compiled = transpile(qc, backend)
        job_sim = backend.run(compiled, shots=1)
        result_sim = job_sim.result()
        counts = result_sim.get_counts()

        if not counts:
            continue

        counts_key = list(counts.keys())[0]
        alice_bit = int(counts_key[0])
        bob_bit = int(counts_key[1])

        if (alice_basis, bob_basis) in shared_basis:
            key_alice.append(alice_bit)
            key_bob.append(1 - bob_bit)  

    print("alice key:", key_alice)
    print("bob's key:  ", key_bob)
    return key_alice, key_bob

print(f"N = {N} rounds = {rounds}")
S_value = bell_inequality(rounds)
if S_value >= 2.8:
    print("secured")
elif (2.5 < S_value < 2.8):
    print("might be eavesdropper or there're some noise")
else:
    print("definitely eavesdropper")
alice_key, bob_key = key(rounds)


N = 100 rounds = 450
S = 2.84000
secured
alice key: [1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0]
bob's key:   [1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0]
